In [12]:
import os
import threading
import requests

op = webdriver.ChromeOptions()
# op.add_argument('--headless=new')
prefs = {
    'profile.default_content_settings.popups': 0,
    'download.default_directory': r"/home/administrator/cbs_bag_hold/data",
    'directory_upgrade': True
}
op.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(options=op)

session = HTMLSession()

def login_flo():
    driver.get("http://10.24.1.71/mh-ops")
    time.sleep(5)
    username = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[1]")
    username.send_keys("ca.2670054")

    password = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[2]")
    password.send_keys("Chauhan@8091")
    time.sleep(2)
    try:
        cross = driver.find_element(By.XPATH, "/html/body/div[4]/div/button")
        cross.click()
    except:
        print("Cross Button Failed")
    time.sleep(1)
    submit = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/form/div/div[4]/div[4]/button/span")
    submit.click()
    time.sleep(3)

login_flo()
session_cookie = driver.get_cookies()
csrf_token = driver.execute_script("return document.querySelector('meta[name=csrf-token]').getAttribute('content');")

if csrf_token:
    session.headers.update({"csrf-token": csrf_token})
    print(csrf_token)
else:
    print("CSRF Token not found in page")

selenium_user_agent = driver.execute_script("return navigator.userAgent;")
print(selenium_user_agent)
session.headers.update({"user-agent": selenium_user_agent})
for cookie in driver.get_cookies():
    session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain']) # Can't is set this here

with open("a.txt", 'r') as f:
    shipments = [line.strip() for line in f if line.strip()]

print(len(shipments))

batch_size = 3000
batches = [shipments[i:i+batch_size] for i in range(0, len(shipments), batch_size)]

def process_batch(batch):
    url = "http://10.24.1.71/mh-ops-routes-api/mh-ops-proxy/v1/track-and-trace-proxy/shipment/bulk-track/generate-request-id"
    response = session.post(url, json={'shipment_ids': batch})

    if response.status_code == 200:
        response_json = response.json()
        request_tracking_id = response_json["data"]["request_tracking_id"]
        print(f"Request tracking ID for batch {batch}: {request_tracking_id}")
      
        csv_url = f"http://10.24.1.71/mh-ops-routes-api/mh-ops-proxy/v1/track-and-trace-proxy/shipment/bulk-download/{request_tracking_id}"
        csv_response = session.get(csv_url)

        if csv_response.status_code == 200:
            download_directory = "/home/flipkart/auto_print/input"
            with open(os.path.join(download_directory, f"{request_tracking_id}.csv"), "wb") as csv_file:
                csv_file.write(csv_response.content)
            print(f"CSV file for batch {batch} downloaded successfully")
        else:
            print(f"Failed to download CSV file for batch {batch}. Status code: {csv_response.status_code}")
    else:
        print(f"Failed to generate request ID for batch {batch}. Status code: {response.status_code}")


threads = []
for batch in batches:
    thread = threading.Thread(target=process_batch, args=(batch,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

print("All batches processed.")


Cross Button Failed
LBvh4zva-EeeuaoSt6klEvtOAUz5q8_lt-WM
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36
3000
Request tracking ID for batch ['FMPC3431056830', 'FMPC3433403401', 'FMPC3434064105', 'FMPC3433659447', 'FMPC3434008544', 'FMPP2139286665', 'FMPC3433171827', 'FMPP2140328476', 'FMPC3433259210', 'FMPC3433825041', 'FMPC3431392678', 'FMPC3432776809', 'FMPP2138913963', 'FMPC3433457431', 'FMPC3433332660', 'FMPP2139800520', 'FMPC3431325242', 'FMPC3433263843', 'NAPC0011091424', 'FMPP2140493333', 'FMPC3433126388', 'FMPC3430156143', 'OIPC0013808596', 'FMPC3429543475', 'FMPC3430422462', 'FMPP2138911436', 'FMPP2141363846', 'FMPP2140837691', 'FMPP2140278515', 'FMPC3430273003', 'MYEC1006060836', 'FMPC3433416910', 'FMPC3431269162', 'FMPC3432956893', 'FMPP2139375702', 'FMPP2140035670', 'FMPP2140615660', 'FMPC3433877068', 'FMPC3428204212', 'FMPP2137378659', 'FMPP2140982656', 'FMPC3432311258', 'FMPP2141299962', 'FMPC3431195404', 'FMPC34336290

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_109010/963787430.py", line 74, in process_batch
  File "/home/flipkart/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 284, in _modified_open
    return io_open(file, *args, **kwargs)
OSError: [Errno 36] File name too long: "/home/flipkart/auto_print/input/downloaded_file_['FMPC3431056830', 'FMPC3433403401', 'FMPC3434064105', 'FMPC3433659447', 'FMPC3434008544', 'FMPP2139286665', 'FMPC3433171827', 'FMPP2140328476', 'FMPC3433259210', 'FMPC3433825041', 'FMPC3431392678', 'FMPC3432776809', 'FMPP2138913963', 'FMPC3433457431', 'FMPC3433332660', 'FMPP2139800520', 'FMPC3431325242', 'FMPC3433263843', 'NAPC0011091424', 'FMPP2140493333', 'FMPC3433126388', 'FMPC3430156143', 'OIPC0013808596', 'FMPC

All batches processed.
